# Clustering Crypto

In [45]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [46]:
# Load the crypto_data.csv dataset.

file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path)

crypto_df.head(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015,2015 coin,X11,True,PoW/PoS,NaN,0
7,BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [47]:
# Keep all the cryptocurrencies that are being traded.

crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [48]:
# Keep all the cryptocurrencies that have a working algorithm.
pd.isna(crypto_df['Algorithm'])

0       False
1       False
2       False
3       False
4       False
        ...  
1243    False
1244    False
1245    False
1246    False
1247    False
Name: Algorithm, Length: 1144, dtype: bool

In [49]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns ='IsTrading')

In [50]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()

In [51]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df.TotalCoinsMined > 0]
crypto_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [39]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = crypto_df.filter(['CoinName'], axis=1)
crypto_names

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum
...,...
1238,ZEPHYR
1242,Gapcoin
1245,Beldex
1246,Horizen


In [40]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(['CoinName'],axis = 1)
crypto_df

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [41]:
# Drop the 'Unnamed' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(['Unnamed: 0'],axis = 1)
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2000000000
1242,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,CryptoNight,PoW,9.802226e+08,1400222610
1246,Equihash,PoW,7.296538e+06,21000000


In [42]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
print(X.shape)
X.head(10)

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
0,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
11,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Standardize the data with StandardScaler().
scaler = StandardScaler()
crypto_scaled = scaler.fit_transform(X)
crypto_scaled[0:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

### Deliverable 2: Reducing Data Dimensions Using PCA

In [53]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
df_pca = pca.fit_transform(crypto_scaled)
df_pca

array([[-0.33878124,  0.89551833, -0.49099754],
       [-0.32207803,  0.89541164, -0.49133495],
       [ 2.31000092,  1.51395471, -0.60818675],
       ...,
       [ 0.32590907, -2.22812024,  0.383752  ],
       [-0.15884248, -1.9638668 ,  0.39760567],
       [-0.28480877,  0.82893819, -0.25936258]])

In [59]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data = df_pca, columns = ['PC1','PC2','PC3'], index = crypto_df.index
)
pcs_df

,PC1,PC2,PC3
0,-0.338781,0.895518,-0.490998
2,-0.322078,0.895412,-0.491335
5,2.310001,1.513955,-0.608187
7,-0.151912,-1.244954,0.167230
8,-0.150386,-1.942884,0.321454
...,...,...,...
1238,2.434864,0.987196,0.092009
1242,-0.336821,0.895380,-0.491025
1245,0.325909,-2.228120,0.383752
1246,-0.158842,-1.963867,0.397606


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [60]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE

inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(pcs_df)
   inertia.append(km.inertia_)

elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k",y="inertia",xticks=k,title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [61]:
# Initialize the K-means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)


In [66]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
cluster_df = pd.concat([crypto_df,pcs_df],axis =1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
cluster_df['CoinName'] = crypto_names.CoinName

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
cluster_df["Class"]= model.labels_

# Print the shape of the clustered_df
print(cluster_df.shape)
cluster_df.head(10)

(532, 10)


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.338781,0.895518,-0.490998,1
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.322078,0.895412,-0.491335,1
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.310001,1.513955,-0.608187,1
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.151912,-1.244954,0.167230,0
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.150386,-1.942884,0.321454,0
9,LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000,-0.169022,-1.159877,0.019372,0
10,DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000,-0.389304,1.132359,-0.477641,1
11,XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0,-0.149155,-2.151656,0.410331,0
12,ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000,-0.148824,-1.943001,0.321433,0
13,ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000,-0.158841,-1.963867,0.397606,0


In [67]:
cluster_df = cluster_df.drop(['Unnamed: 0'],axis = 1)
cluster_df.head(5)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.338781,0.895518,-0.490998,1
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.322078,0.895412,-0.491335,1
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.310001,1.513955,-0.608187,1
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.151912,-1.244954,0.167230,0
8,Ethereum,Ethash,PoW,1.076842e+08,0,-0.150386,-1.942884,0.321454,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [69]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    cluster_df,
    hover_name="CoinName",
    hover_data=["Algorithm"],
    x="PC1",
    y="PC2",
    z="PC3",
    color="Class",
    symbol="Class",
    width=700,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [70]:
# Create a table with tradable cryptocurrencies.
cluster_df.hvplot.table(sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class]

In [75]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {len(cluster_df.index)} tradable cryptos")

There are 532 tradable cryptos


In [77]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_cluster = cluster_df[['TotalCoinSupply', 'TotalCoinsMined']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [79]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_df = pd.DataFrame(
    data = X_cluster_scaled,columns = ["TotalCoinSupply","TotalCoinsMined"], index = cluster_df.index
)
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
new_df = pd.concat([new_df,cluster_df['CoinName']],axis =1)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = pd.concat([new_df,cluster_df["Class"]],axis =1)
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.200000e-11,0.000000,42 Coin,1
2,5.320000e-04,0.001066,404Coin,1
5,3.141593e-01,0.029576,EliteCoin,1
7,2.100000e-05,0.000018,Bitcoin,0
8,0.000000e+00,0.000109,Ethereum,0
9,8.400000e-05,0.000064,Litecoin,0
10,2.200000e-05,0.000009,Dash,1
11,0.000000e+00,0.000017,Monero,0
12,2.100000e-04,0.000115,Ethereum Classic,0
13,2.100000e-05,0.000007,ZCash,0


In [80]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", by="Class",hover_cols = ["CoinName"])

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)